<a href="https://colab.research.google.com/github/podobongbong/the_project/blob/main/Nov18_data_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/podobongbong/the_project.git
import os
os.chdir('/content/the_project')
import pandas as pd


fatal: destination path 'the_project' already exists and is not an empty directory.


In [2]:

# Energy Data (2004-2023)
energy_data = pd.read_csv('energy.csv')
energy_data.columns = ['year','city','district','type','jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

#주택용
energy_data = energy_data[energy_data['type']=='주택용']
monthly_columns = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
aggregated_data = energy_data.groupby(['year','city'])[monthly_columns].sum().reset_index()

energy_data['city'] = energy_data['city'].replace({'서울특별시': 'seoul', '부산광역시': 'busan'})
aggregated_energy_data = energy_data.groupby(['year', 'city'])[monthly_columns].sum().reset_index()

energy_seoul = aggregated_energy_data[aggregated_energy_data['city'] == 'seoul']
energy_busan = aggregated_energy_data[aggregated_energy_data['city'] == 'busan']

# Population Data (2008-2023)
population_data = pd.read_csv('/content/the_project/population.csv')
population_data.columns = ['year', 'city', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
population_data['city'] = population_data['city'].replace({'seoul': 'seoul', 'busan': 'busan'})

In [3]:
# Urban Data (2014.07 -2023.12)

urban_data = pd.read_csv('/content/the_project/urban_char.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/the_project/urban_char.csv'

In [ ]:
# Cliamte Data (2004-2023)
climate_busan = pd.read_csv('/content/the_project/climate_busan.csv')
climate_seoul = pd.read_csv('/content/the_project/climate_seoul.csv')
climate_data = pd.concat([climate_busan, climate_seoul], ignore_index=True)
df= climate_data.copy()

df['temp_range'] = df['high'] - df['low']
df['high_deviation'] = df['high'] - df['average']
df['low_deviation'] = df['average'] - df['low']
df = df.dropna()

## Climate feature selection: Primary Component Analysis
from sklearn.decomposition import PCA
import pandas as pd
climate_vars = df[['average', 'low', 'high', 'temp_range','high_deviation','low_deviation']]
climate_vars = climate_vars.dropna()

## Apply PCA to reduce to 1 or 2 components
pca = PCA(n_components=1)  # Adjust to 1 or 2 based on analysis
climate_pca = pca.fit_transform(climate_vars)
df['climate_summary'] = climate_pca[:, 0]  # Only the first component
df['city'] = df['station'].replace({108: 'seoul', 159: 'busan'})
df['city'] = df['city'].astype(str)

climate = df[['ym','city','climate_summary']]


In [ ]:

e_seoul = energy_seoul[monthly_columns].values.flatten().reshape(-1, 1)
e_busan = energy_busan[monthly_columns].values.flatten().reshape(-1, 1)

p_seoul = population_data[population_data['city'] == 'seoul'][monthly_columns].values.flatten().reshape(-1, 1)
p_busan = population_data[population_data['city'] == 'busan'][monthly_columns].values.flatten().reshape(-1, 1)


In [ ]:
c_vars = ['ym','average', 'low', 'high', 'temp_range','high_deviation','low_deviation','climate_summary']
seoul = df[df['city']=='seoul'][c_vars]
busan = df[df['city']=='busan'][c_vars]

In [ ]:
seoul['energy']=e_seoul
busan['energy']=e_busan

In [ ]:
seoul_08 = seoul.iloc[48:240].reset_index(drop=True)
busan_08 = busan.iloc[48:240].reset_index(drop=True)
seoul_08['pop'] = p_seoul[0:192]
busan_08['pop'] = p_busan[0:192]

In [ ]:
# prompt: seoul_08 exploratory data analysis


# Descriptive Statistics
print(seoul_08.describe())

import matplotlib.pyplot as plt
import seaborn as sns
# Convert the 'ym' column to datetime objects
seoul_08['ym'] = pd.to_datetime(seoul_08['ym'].str.strip(), format='%Y-%m')

# Extract the month as a numerical feature for correlation analysis
seoul_08['month'] = seoul_08['ym'].dt.month

# Correlation Matrix (excluding the original 'ym' column)
correlation_matrix = seoul_08.drop(columns=['ym']).corr()  # Exclude 'ym'
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Seoul Energy Consumption and Climate Variables (2008-2023)')
plt.show()

# Time Series Plots for all variables in seoul_08
for column in seoul_08.columns:
  if column not in ['ym', 'month']:  # Exclude 'ym' and 'month' columns
    plt.figure(figsize=(12, 6))
    plt.plot(seoul_08['ym'], seoul_08[column])
    plt.xlabel('Month')
    plt.ylabel(column)
    plt.title(f'Seoul {column} Trend (2008-2023)')
    plt.xticks(rotation=45)
    plt.show()


In [ ]:
# prompt: busan_08 exploratory data analysis

# Descriptive Statistics
print(busan_08.describe())

import matplotlib.pyplot as plt
import seaborn as sns
# Convert the 'ym' column to datetime objects
busan_08['ym'] = pd.to_datetime(busan_08['ym'].str.strip(), format='%Y-%m')

# Extract the month as a numerical feature for correlation analysis
busan_08['month'] = busan_08['ym'].dt.month

# Correlation Matrix (excluding the original 'ym' column)
correlation_matrix = busan_08.drop(columns=['ym']).corr()  # Exclude 'ym'
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Busan Energy Consumption and Climate Variables (2008-2023)')
plt.show()



# Time Series Plots for all variables in busan_08
for column in busan_08.columns:
  if column not in ['ym', 'month']:  # Exclude 'ym' and 'month' columns
    plt.figure(figsize=(12, 6))
    plt.plot(busan_08['ym'], busan_08[column])
    plt.xlabel('Month')
    plt.ylabel(column)
    plt.title(f'Busan {column} Trend (2008-2023)')
    plt.xticks(rotation=45)
    plt.show()



In [ ]:
#부산, 서울 모두 2010년 10월 인구 통계가 뜀: 거주불명자(2010년 10월부터 통계포함)로 인한 것으로 보임. 거주불명자 인구 따로 제공 되지 않음